# RNN from scratch

![](RNN.png)

In [1]:
import tensorflow as tf
import numpy as np

In [2]:
class MyRNNcell(tf.keras.layers.Layer):
    def __init__(self,rnn_units,input_dim,output_dim):
        
        super(MyRNNcell,self).__init__()
        
        #Matrices de Pesos
        self.W_xh=self.add_weight([rnn_units,input_dim])
        self.W_hh=self.add_weight([rnn_units,rnn_units])
        self.W_yh=self.add_weight([output_dim,rnn_units])
        
        #inicializar h
        
        self.h=tf.zeros([rnn_units,1])
        
    def call(self,x):
        self.h=tf.math.tanh(self.W_xh*x+self.W_hh*self.h)
        
        #output
        output=self.W_yh*self.h
        
        return(output,self.h)
        

In [3]:
#MyRNNcell(5,4,1).call

![](LSTM1.png)



# Forget gate
First, we have the forget gate. This gate decides what information should be thrown away or kept. Information from the previous hidden state and information from the current input is passed through the sigmoid function. Values come out between 0 and 1. The closer to 0 means to forget, and the closer to 1 means to keep.

![](LSTM2.gif)

# Input Gate
To update the cell state, we have the input gate. First, we pass the previous hidden state and current input into a sigmoid function. That decides which values will be updated by transforming the values to be between 0 and 1. 0 means not important, and 1 means important. You also pass the hidden state and current input into the tanh function to squish values between -1 and 1 to help regulate the network. Then you multiply the tanh output with the sigmoid output. The sigmoid output will decide which information is important to keep from the tanh output.

![](LSTM3.gif)

# Cell State
Now we should have enough information to calculate the cell state. First, the cell state gets pointwise multiplied by the forget vector. This has a possibility of dropping values in the cell state if it gets multiplied by values near 0. Then we take the output from the input gate and do a pointwise addition which updates the cell state to new values that the neural network finds relevant. That gives us our new cell state.

![](LSTM4.gif)

# Output Gate
Last we have the output gate. The output gate decides what the next hidden state should be. Remember that the hidden state contains information on previous inputs. The hidden state is also used for predictions. First, we pass the previous hidden state and the current input into a sigmoid function. Then we pass the newly modified cell state to the tanh function. We multiply the tanh output with the sigmoid output to decide what information the hidden state should carry. The output is the hidden state. The new cell state and the new hidden is then carried over to the next time step.

![](LSTM5.gif)

## Función LSTM

In [4]:
import numpy as np
import math
def sigm(x):
    return(1/(1+np.exp(-x)))

In [5]:
def LSTM(h_prev,c_prev,x):
    f=sigm(h_prev+x) #1.Forget Gate
    i=sigm(h_prev+x)#2. input
    c=math.tanh(h_prev+x)#2. input
    c=f*c_prev+i*c#3.cell state
    o=sigm(h_prev+x)
    h=o+math.tanh(c)
    return(h,c)

In [6]:
LSTM(3.5677,8.45,4)

(1.9994833746149006, 9.445117474235419)

# LSTM from scratch

![](LSTM6.png)

**Notar que el esquema es una celda simple**

**feedforward**


$Z_{t}=[x_{t}+h_{t-1}]$ 

$a_{f}=W_{f}Z_{t}$

$f_{t}=\sigma(a_{f}+b_{f})$


$a_{i}=W_{i}Z_{t}$

$i_{t}=\sigma(a_{i}+b_{i})$

$a_{c}=W_{c}Z_{t}$

$c_{t}^{*}=Tanh(a_{c}+b_{c})$

$a_{o}=W_{o}Z_{t}$

$o_{t}=\sigma(a_{o}+b_{o})$

$c_{t}=f_{t}c_{t-1}+i_{t}c_{t}^{*}$

$h_{t}=o_{t}Tanh(c_{t})$



$v_{t}=W_{v}h_{t}+b_{v}$

$y_{t}^{*}=softmax(v_{t})$

In [22]:
class LSTM:
    
    def __init__(self,char_to_idx, idx_to_char,nn_units,input_dim,output_dim,seq_len=25,beta1=0.9, beta2=0.999,epochs=10,lr=0.01):
        
        self.epochs=epochs
        self.char_to_idx=char_to_idx
        self.idx_to_char=idx_to_char
        self.nn_units=nn_units
        self.input_dim=input_dim
        self.seq_len=seq_len
        self.output_dim=output_dim
        self.beta1=beta1
        self.beta2=beta2
        self.lr=lr
        
        self.params={}
        
        self.params['Wf']=np.random.randn(self.nn_units,self.nn_units+self.input_dim)*(1.0/np.sqrt(self.nn_units+self.input_dim))
        self.params['bf']=np.ones((self.nn_units,1))
        
        self.params['Wi']=np.random.randn(self.nn_units,self.nn_units+self.input_dim)*(1.0/np.sqrt(self.nn_units+self.input_dim))
        self.params['bi']=np.zeros((self.nn_units,1))
        
        self.params['Wc']=np.random.randn(self.nn_units,self.nn_units+self.input_dim)*(1.0/np.sqrt(self.nn_units+self.input_dim))
        self.params['bc']=np.zeros((self.nn_units,1))
        
        self.params['Wo']=np.random.randn(self.nn_units,self.nn_units+self.input_dim)*(1.0/np.sqrt(self.nn_units+self.input_dim))
        self.params['bo']=np.zeros((self.nn_units,1))
        
        self.params['Wv']=np.random.randn(self.input_dim,self.nn_units)*(1.0/np.sqrt(self.input_dim))
        self.params['bv']=np.zeros((self.input_dim,1))
        
        self.grads={}
        self.adams={}
        #gradientes y adams
        for key in self.params.keys():
            self.grads['d'+key]=np.zeros_like(self.params[key])
            self.adams['m'+key]=np.zeros_like(self.params[key])
            self.adams['v'+key]=np.zeros_like(self.params[key])
            
        self.smooth_loss=-np.log(1.0/self.input_dim)*self.seq_len
        
        return
    
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    #LSTM.sigmoid = sigmoid
    LSTM.sigmoid=sigmoid


    def softmax(self, x):
        e_x = np.exp(x - np.max(x)) # max(x) subtracted for numerical stability
        return e_x / np.sum(e_x)

    def sqr(self, x):
        if x<0:
            return(0)
        else:
            return(np.sqrt(x))
        
    
    
    #LSTM.softmax = softmax
    LSTM.softmax=softmax
    LSTM.sqr=sqr
    
    def clip_grads(self):
        for key in self.grads:
            np.clip(self.grads[key], -5, 5, out=self.grads[key])
        return

    #LSTM.clip_grads = clip_grads
    LSTM.clip_grads=clip_grads


    def reset_grads(self):
        for key in self.grads:
            self.grads[key].fill(0)
        return

    #LSTM.reset_grads = reset_grads
    LSTM.reset_grads=reset_grads
    
    def update_params(self,batch_num):
        for key in self.params.keys():
            self.adams['m'+key]=self.adams['m'+key]*self.beta1+self.grads['d'+key]*(1-self.beta1)
            self.adams['v'+key]=self.adams['v'+key]*self.beta2+self.grads['d'+key]*(1-self.beta2)
            
            m_correlated = self.adams["m" + key] / (1 - self.beta1**batch_num)
            v_correlated = self.adams["v" + key] / (1 - self.beta2**batch_num)
            
            mifun=np.vectorize(lambda x: 0 if x<0 else np.sqrt(x))
            #print(v_correlated)
            self.params[key] -= self.lr * m_correlated / (mifun(v_correlated) + 1e-8)  
        return 
    LSTM.update_params=update_params
    
    def forward_step(self, x, h_prev, c_prev):
        z = np.row_stack((h_prev, x))

        f = self.sigmoid(np.dot(self.params["Wf"], z) + self.params["bf"])
        i = self.sigmoid(np.dot(self.params["Wi"], z) + self.params["bi"])
        c_bar = np.tanh(np.dot(self.params["Wc"], z) + self.params["bc"])

        c = f * c_prev + i * c_bar
        o = self.sigmoid(np.dot(self.params["Wo"], z) + self.params["bo"])
        h = o * np.tanh(c)

        v = np.dot(self.params["Wv"], h) + self.params["bv"]
        y_hat = self.softmax(v)
        return y_hat, v, h, o, c, c_bar, i, f, z

    LSTM.forward_step = forward_step
   
    def backward_step(self, y, y_hat, dh_next, dc_next, c_prev, z, f, i, c_bar, c, o, h):
        dv=np.copy(y_hat)
        dv[y] -= 1 # yhat - y
        
        self.grads['dWv']+=np.dot(dv,h.T)
        self.grads['dbv']+=dv
        
        #en 7.1 podemos ver que todos los gradientes usan el gradiente de h dh por tanto hay que calcular explícitamente
        dh=np.dot(self.params['Wv'].T,dv)
        dh+=dh_next
        
        #Ahora queremos los gradientes de los W's y B's:
        self.grads['dWo']+=np.dot(dh*np.tanh(c)*o*(1-o),z.T)
        self.grads['dbo']+=dh*np.tanh(c)*o*(1-o)
        
        #usando en términos de dc...
        dc=dh*o*(1-np.tanh(c)**2)
        dc+=dc_next
        
        self.grads['dWc']+=np.dot(dc*i*(1-c_bar**2),z.T)
        self.grads['dbc']+=dc*i*(1-c_bar**2)
        
        self.grads['dWi']+=np.dot(dc*c_bar*i*(1-i),z.T)
        self.grads['dbi']+=dc*c_bar*i*(1-i)
        
        self.grads['dWf']+=np.dot(dc*c_prev*f*(1-f),z.T)
        self.grads['dbf']+=dc*c_prev*f*(1-f)
        
        dz=np.dot(self.params['Wf'].T,dc*c_prev*f*(1-f))+np.dot(self.params['Wi'].T,dc*c_bar*i*(1-i))+np.dot(self.params['Wc'].T,dc*i*(1-c_bar**2))+np.dot(self.params['Wo'].T,dh*np.tanh(c)*o*(1-o))
        
        dh_prev=dz[:self.nn_units,:]
        dc_prev=f*dc
        
        return dh_prev, dc_prev
    LSTM.backward_step=backward_step   
    
    def forward_backward(self, x_batch, y_batch, h_prev, c_prev):
        x, z = {}, {}
        f, i, c_bar, c, o = {}, {}, {}, {}, {}
        y_hat, v, h = {}, {}, {}

        # Values at t= - 1
        h[-1] = h_prev
        c[-1] = c_prev

        loss = 0
        for t in range(self.seq_len): 
            x[t] = np.zeros((self.input_dim, 1))
            x[t][x_batch[t]] = 1

            y_hat[t], v[t], h[t], o[t], c[t], c_bar[t], i[t], f[t], z[t] = \
            self.forward_step(x[t], h[t-1], c[t-1])

            loss += -np.log(y_hat[t][y_batch[t],0])

        self.reset_grads()

        dh_next = np.zeros_like(h[0])
        dc_next = np.zeros_like(c[0])

        for t in reversed(range(self.seq_len)):
            dh_next, dc_next = self.backward_step(y_batch[t], y_hat[t], dh_next, 
                                                  dc_next, c[t-1], z[t], f[t], i[t], 
                                                  c_bar[t], c[t], o[t], h[t]) 
        return loss, h[self.seq_len-1], c[self.seq_len-1]

    LSTM.forward_backward = forward_backward

    def sample(self, h_prev, c_prev, sample_size):
        x = np.zeros((self.input_dim, 1))
        h = h_prev
        c = c_prev
        sample_string = "" 

        for t in range(sample_size):
            y_hat, _, h, _, c, _, _, _, _ = self.forward_step(x, h, c)        

            # get a random index within the probability distribution of y_hat(ravel())
            idx = np.random.choice(range(self.input_dim), p=y_hat.ravel())
            x = np.zeros((self.input_dim, 1))
            x[idx] = 1

            #find the char with the sampled index and concat to the output string
            char = self.idx_to_char[idx]
            sample_string += char
        return sample_string

    LSTM.sample = sample  
    
    def gradient_check(self, x, y, h_prev, c_prev, num_checks=10, delta=1e-6):
        """
        Checks the magnitude of gradients against expected approximate values
        """
        print("**********************************")
        print("Gradient check...\n")

        _, _, _ = self.forward_backward(x, y, h_prev, c_prev)
        grads_numerical = self.grads

        for key in self.params:
            print("---------", key, "---------")
            test = True

            dims = self.params[key].shape
            grad_numerical = 0
            grad_analytical = 0

            for _ in range(num_checks):  # sample 10 neurons

                idx = int(np.random.uniform(0, self.params[key].size))
                old_val = self.params[key].flat[idx]

                self.params[key].flat[idx] = old_val + delta
                J_plus, _, _ = self.forward_backward(x, y, h_prev, c_prev)

                self.params[key].flat[idx] = old_val - delta
                J_minus, _, _ = self.forward_backward(x, y, h_prev, c_prev)

                self.params[key].flat[idx] = old_val

                grad_numerical += (J_plus - J_minus) / (2 * delta)
                grad_analytical += grads_numerical["d" + key].flat[idx]

            grad_numerical /= num_checks
            grad_analytical /= num_checks

            rel_error = abs(grad_analytical - grad_numerical) / abs(grad_analytical + grad_numerical)

            if rel_error > 1e-2:
                if not (grad_analytical < 1e-6 and grad_numerical < 1e-6):
                    test = False
                    assert (test)

            print('Approximate: \t%e, Exact: \t%e =>  Error: \t%e' % (grad_numerical, grad_analytical, rel_error))
        print("\nTest successful!")
        print("**********************************\n")
        return

    def train(self, X, verbose=True):
        """
        Main method of the LSTM class where training takes place
        """
        J = []  # to store losses

        num_batches = len(X) // self.seq_len
        X_trimmed = X[: num_batches * self.seq_len]  # trim input to have full sequences

        for epoch in range(self.epochs):
            print(f'epoch:{epoch}')
            h_prev = np.zeros((self.nn_units, 1))
            c_prev = np.zeros((self.nn_units, 1))

            for j in range(0, len(X_trimmed) - self.seq_len, self.seq_len):
                # prepare batches
                x_batch = [self.char_to_idx[ch] for ch in X_trimmed[j: j + self.seq_len]]
                y_batch = [self.char_to_idx[ch] for ch in X_trimmed[j + 1: j + self.seq_len + 1]]

                loss, h_prev, c_prev = self.forward_backward(x_batch, y_batch, h_prev, c_prev)

                # smooth out loss and store in list
                self.smooth_loss = self.smooth_loss * 0.999 + loss * 0.001
                J.append(self.smooth_loss)

                # check gradients
                if epoch == 0 and j == 0:
                    self.gradient_check(x_batch, y_batch, h_prev, c_prev, num_checks=10, delta=1e-7)

                self.clip_grads()

                batch_num = epoch * self.epochs + j / self.seq_len + 1
                self.update_params(batch_num)

                # print out loss and sample string
                if verbose:
                    if j % 400000 == 0:
                        print('Epoch:', epoch, '\tBatch:', j, "-", j + self.seq_len,
                              '\tLoss:', round(self.smooth_loss, 2))
                        s = self.sample(h_prev, c_prev, sample_size=250)
                        print(s, "\n")

        return J, self.params    

![](lstm_bp.png)

In [18]:
#np.row_stack(([1,2,3],[2,4,5]))

![](lstm_bp.png)

![](lstm_bp.png)

# Backpropagation

![](lstm_bp.jpeg)

$J=\frac{1}{2}(y_{t}^{*}-y_{t})^{2}$

$\frac{\partial J}{\partial V_{t}}=y_{t}^{*}-y_{t}$

$\frac{\partial J}{\partial W_{v}}=\frac{\partial J}{\partial V_{t}}\frac{\partial V_{t}}{\partial W_{v}}=(y_{t}^{*}-y_{t})*h_{t}$

$\frac{\partial J}{\partial h_{t}}=\frac{\partial J}{\partial V_{t}}\frac{\partial V_{t}}{\partial h_{t}}=(y_{t}^{*}-y_{t})*W_{v}$

$\frac{\partial J}{\partial b_{v}}=\frac{\partial J}{\partial V_{t}}\frac{\partial V_{t}}{\partial b_{v}}=(y_{t}^{*}-y_{t})$

$\frac{\partial J}{\partial o_{t}}=\frac{\partial J}{\partial h_{t}}\frac{\partial h_{t}}{\partial o{t}}=\frac{\partial J}{\partial h_{t}}*Tanh(c_{t})$

$\frac{\partial J}{\partial c_{t}}=\frac{\partial J}{\partial h_{t}}\frac{\partial h_{t}}{\partial c{t}}=\frac{\partial J}{\partial h_{t}}*o_{t}(1-Tanh^2(c_{t}))$

$\frac{\partial J}{\partial a_{o}}=\frac{\partial J}{\partial o_{t}}\frac{\partial o_{t}}{\partial a{o}}=\frac{\partial J}{\partial o_{t}}*\sigma(a_{o}+b_{o})(1-\sigma(a_{o}+b_{o}))=\frac{\partial J}{\partial h_{t}}*Tanh(c_{t})*o_{t}(1-o_{t})$

$\frac{\partial J}{\partial c_{t}^{*}}=\frac{\partial J}{\partial c_{t}}\frac{\partial c_{t}}{\partial c_{t}^{*}}=\frac{\partial J}{\partial c_{t}}\frac{\partial (f_{t}c_{t-1}+i_{t}c_{t}^{*})}{\partial c_{t}^{*}}=\frac{\partial J}{\partial c_{t}}*i_{t}$

$\frac{\partial J}{\partial a_{c}}=\frac{\partial J}{\partial c_{t}^{*}}\frac{\partial c_{t}^{*}}{\partial a_{c}}=\frac{\partial J}{\partial c_{t}^{*}}\frac{\partial (Tanh(a_{c}+b_{c}))}{\partial a_{c}}=\frac{\partial J}{\partial c_{t}^{*}}*(1-Tanh^2(a_{c}+b_{c}))=\frac{\partial J}{\partial c_{t}}*i_{t}*(1-Tanh^2(a_{c}+b_{c}))=\frac{\partial J}{\partial c_{t}^{*}}*(1-Tanh^2(a_{c}+b_{c}))=\frac{\partial J}{\partial c_{t}}*i_{t}*(1-Tanh^2(a_{c}+b_{c}))=\frac{\partial J}{\partial c_{t}}*i_{t}*(1-c_{t}^{*2})$


$\frac{\partial J}{\partial i_{t}}=\frac{\partial J}{\partial c_{t}}\frac{\partial c_{t}}{\partial i_{t}}=\frac{\partial J}{\partial c_{t}}\frac{\partial (f_{t}c_{t-1}+i_{t}c_{t}^{*})}{\partial i_{t}}=\frac{\partial J}{\partial c_{t}}*c_{t}^{*}$

$\frac{\partial J}{\partial a_{i}}=\frac{\partial J}{\partial i_{t}}\frac{\partial i_{t}}{\partial a_{i}}=\frac{\partial J}{\partial i_{t}}\frac{\partial (\sigma(a_{i}+b_{i}))}{\partial a_{i}}=\frac{\partial J}{\partial i_{t}}\sigma(a_{i}+b_{i})(1-\sigma(a_{i}+b_{i}))=\frac{\partial J}{\partial c_{t}}*c_{t}^{*}*i_{t}*(1-i_{t})$

$\frac{\partial J}{\partial f_{t}}=\frac{\partial J}{\partial c_{t}}\frac{\partial c_{t}}{\partial f_{t}}=\frac{\partial J}{\partial c_{t}}\frac{\partial (f_{t}c_{t-1}+i_{t}c_{t}^{*})}{\partial f_{t}}=\frac{\partial J}{\partial c_{t}}*c_{t-1}$

$\frac{\partial J}{\partial a_{f}}=\frac{\partial J}{\partial f_{t}}\frac{\partial f_{t}}{\partial a_{f}}=\frac{\partial J}{\partial f_{t}}\frac{\partial \sigma(a_{f}+b_{f})}{\partial a_{f}}=\frac{\partial J}{\partial f_{t}}\sigma(a_{f}+b_{f})*(1-\sigma(a_{f}+b_{f}))=\frac{\partial J}{\partial c_{t}}*c_{t-1}*f_{t}*(1-f_{t})$

$\frac{\partial J}{\partial c_{t-1}}=\frac{\partial J}{\partial c_{t}}\frac{\partial c_{t}}{\partial c_{t-1}}=\frac{\partial J}{\partial c_{t}}f_{t}$

$\frac{\partial J}{\partial Z_{t}}=\frac{\partial J}{\partial a_{f}}\frac{\partial a_{f}}{\partial Z_{t}}+\frac{\partial J}{\partial a_{i}}\frac{\partial a_{i}}{\partial Z_{t}}+\frac{\partial J}{\partial a_{c}}\frac{\partial a_{c}}{\partial Z_{t}}+\frac{\partial J}{\partial a_{o}}\frac{\partial a_{o}}{\partial Z_{t}}=\frac{\partial J}{\partial a_{f}}W_{f}+\frac{\partial J}{\partial a_{i}}W_{i}+\frac{\partial J}{\partial a_{c}}W_{c}+\frac{\partial J}{\partial a_{o}}W_{o}$

**Todo lo anterior sirvió par armar la regla de la cadena que para tener las derivadas de los $W$'s y $b$'s para hacer el backpropagation de los pesos**

$\frac{\partial J}{\partial W_{f}}=\frac{\partial J}{\partial a_{f}}\frac{\partial a_{f}}{\partial W_{f}}=\frac{\partial J}{\partial a_{f}}*Z_{t}=\frac{\partial J}{\partial h_{t}}*o_{t}(1-Tanh^2(c_{t}))*c_{t-1}*f_{t}*(1-f_{t})*Z_{t}$

$\frac{\partial J}{\partial b_{f}}=\frac{\partial J}{\partial a_{f}}\frac{\partial a_{f}}{\partial b_{f}}=\frac{\partial J}{\partial a_{f}}=\frac{\partial J}{\partial h_{t}}*o_{t}(1-Tanh^2(c_{t}))*c_{t-1}*f_{t}*(1-f_{t})$

$\frac{\partial J}{\partial W_{i}}=\frac{\partial J}{\partial a_{i}}\frac{\partial a_{i}}{\partial W_{i}}=\frac{\partial J}{\partial a_{i}}*Z_{t}=\frac{\partial J}{\partial h_{t}}*o_{t}(1-Tanh^2(c_{t}))*c_{t}^{*}*i_{t}*(1-i_{t})*Z_{t}$

$\frac{\partial J}{\partial b_{i}}=\frac{\partial J}{\partial a_{i}}\frac{\partial a_{i}}{\partial b_{i}}=\frac{\partial J}{\partial a_{i}}=\frac{\partial J}{\partial h_{t}}*o_{t}(1-Tanh^2(c_{t}))*c_{t}^{*}*i_{t}*(1-i_{t})$

$\frac{\partial J}{\partial W_{c}}=\frac{\partial J}{\partial a_{c}}\frac{\partial a_{c}}{\partial W_{c}}=\frac{\partial J}{\partial a_{c}}*Z_{t}=\frac{\partial J}{\partial h_{t}}*o_{t}(1-Tanh^2(c_{t})*i_{t}*(1-c_{t}^{*2})*Z_{t}$

$\frac{\partial J}{\partial b_{c}}=\frac{\partial J}{\partial a_{c}}\frac{\partial a_{c}}{\partial b_{c}}=\frac{\partial J}{\partial a_{c}}=\frac{\partial J}{\partial h_{t}}*o_{t}(1-Tanh^2(c_{t})*i_{t}*(1-c_{t}^{*2})$


$\frac{\partial J}{\partial W_{o}}=\frac{\partial J}{\partial a_{o}}\frac{\partial a_{o}}{\partial W_{o}}=\frac{\partial J}{\partial a_{o}}*Z_{t}=\frac{\partial J}{\partial h_{t}}*Tanh(c_{t})*o_{t}(1-o_{t})*Z_{t}$

$\frac{\partial J}{\partial b_{o}}=\frac{\partial J}{\partial a_{o}}\frac{\partial a_{o}}{\partial b_{o}}=\frac{\partial J}{\partial a_{o}}=\frac{\partial J}{\partial h_{t}}*Tanh(c_{t})*o_{t}(1-o_{t})$

$\frac{\partial J}{\partial W_{v}}=\frac{\partial J}{\partial v_{t}}\frac{\partial v_{t}}{\partial W_{v}}=\frac{\partial J}{\partial v_{t}}*h_{t}=(y_{t}^{*}-y_{t})*h_{t}$

$\frac{\partial J}{\partial b_{v}}=\frac{\partial J}{\partial v_{t}}\frac{\partial v_{t}}{\partial b_{v}}=\frac{\partial J}{\partial v_{t}}=y_{t}^{*}-y_{t}$

## Actualizando pesos

$W_{v}=W_{v}+\frac{\partial J}{\partial v_{t}}*h_{t}^t$

$b_{v}=b_{v}+\frac{\partial J}{\partial v_{t}}$


$W_{o}=W_{o}+\frac{\partial J}{\partial a_{o}}*Z_{t}^t$

$b_{o}=b_{o}+\frac{\partial J}{\partial a_{o}}$


$W_{c}=W_{c}+\frac{\partial J}{\partial a_{c}}*Z_{t}^t$

$b_{c}=b_{c}+\frac{\partial J}{\partial a_{c}}$


$W_{i}=W_{i}+\frac{\partial J}{\partial a_{i}}*Z_{t}^t$

$b_{i}=b_{i}+\frac{\partial J}{\partial a_{i}}$


$W_{f}=W_{f}+\frac{\partial J}{\partial a_{f}}*Z_{t}^t$

$b_{f}=b_{f}+\frac{\partial J}{\partial a_{f}}$


# Forward and backward propagation for all time-steps

The forward and backward propagation steps will be executed within the forward_backward function. Here, we iterate over all time steps and store the results for each time step in dictionaries. In the forward propagation loop, we also accumulate the cross entropy loss.

forward_backward exports the cross entropy loss of the training batch, in addition to the hidden and cell states of the last layer which are fed to the first LSTM cell as hprev and prev of the next training batch.

# Ejecución

In [10]:
data = open('HP1.txt').read().lower()

In [11]:
chars = set(data)
vocab_size = len(chars)
print('data has %d characters, %d unique' % (len(data), vocab_size))

char_to_idx = {w: i for i,w in enumerate(chars)}
idx_to_char = {i: w for i,w in enumerate(chars)}

data has 442767 characters, 56 unique


In [23]:
model = LSTM(char_to_idx, idx_to_char,nn_units=100,input_dim=vocab_size,output_dim=4,seq_len=25,beta1=0.9, beta2=0.999,epochs=10)

J, params = model.train(data)
#print(model.params['Wf'])

epoch:0
**********************************
Gradient check...

--------- Wf ---------
Approximate: 	1.482263e-04, Exact: 	1.482604e-04 =>  Error: 	1.149527e-04
--------- bf ---------
Approximate: 	-8.383147e-03, Exact: 	-8.383183e-03 =>  Error: 	2.153884e-06
--------- Wi ---------
Approximate: 	4.808953e-05, Exact: 	4.805700e-05 =>  Error: 	3.383978e-04
--------- bi ---------
Approximate: 	-5.581747e-03, Exact: 	-5.581705e-03 =>  Error: 	3.729603e-06
--------- Wc ---------
Approximate: 	-7.262798e-03, Exact: 	-7.262768e-03 =>  Error: 	2.062959e-06
--------- bc ---------
Approximate: 	2.172027e-01, Exact: 	2.172026e-01 =>  Error: 	1.201625e-08
--------- Wo ---------
Approximate: 	8.356693e-04, Exact: 	8.356550e-04 =>  Error: 	8.537345e-06
--------- bo ---------
Approximate: 	-1.056150e-02, Exact: 	-1.056156e-02 =>  Error: 	2.807075e-06
--------- Wv ---------
Approximate: 	1.116797e-02, Exact: 	1.116792e-02 =>  Error: 	2.129546e-06
--------- bv ---------
Approximate: 	3.419841e-01, Exact:

C:\Users\Abraham\miniconda3\envs\tf23\lib\site-packages\ipykernel_launcher.py:46: RuntimeWarning: overflow encountered in exp
C:\Users\Abraham\miniconda3\envs\tf23\lib\site-packages\ipykernel_launcher.py:169: RuntimeWarning: divide by zero encountered in log


 r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r r 

Epoch: 0 	Batch: 400000 - 400025 	Loss: inf
                                                                                                                                                                                                                                                           

epoch:1
Epoch: 1 	Batch: 0 - 25 	Loss: inf
rururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururururu 

Epoch: 1 	Batch: 400000 - 400025 	Loss: inf
 ttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttttt

KeyboardInterrupt: 

In [ ]:
#np.sqrt(model.adams['vWf'])

# Versión original

In [24]:
from random import uniform


class LSTM:
    def __init__(self, char_to_idx, idx_to_char, vocab_size, n_h=100, seq_len=25,
                 epochs=10, lr=0.001, beta1=0.9, beta2=0.999):
        """
        Implementation of simple character-level LSTM using Numpy
        """
        self.char_to_idx = char_to_idx  # characters to indices mapping
        self.idx_to_char = idx_to_char  # indices to characters mapping
        self.vocab_size = vocab_size  # no. of unique characters in the training data
        self.n_h = n_h  # no. of units in the hidden layer
        self.seq_len = seq_len  # no. of time steps, also size of mini batch
        self.epochs = epochs  # no. of training iterations
        self.lr = lr  # learning rate
        self.beta1 = beta1  # 1st momentum parameter
        self.beta2 = beta2  # 2nd momentum parameter

        # -----initialise weights and biases-----#
        self.params = {}
        std = (1.0 / np.sqrt(self.vocab_size + self.n_h))  # Xavier initialisation

        # forget gate
        self.params["Wf"] = np.random.randn(self.n_h, self.n_h + self.vocab_size) * std
        self.params["bf"] = np.ones((self.n_h, 1))

        # input gate
        self.params["Wi"] = np.random.randn(self.n_h, self.n_h + self.vocab_size) * std
        self.params["bi"] = np.zeros((self.n_h, 1))

        # cell gate
        self.params["Wc"] = np.random.randn(self.n_h, self.n_h + self.vocab_size) * std
        self.params["bc"] = np.zeros((self.n_h, 1))

        # output gate
        self.params["Wo"] = np.random.randn(self.n_h, self.n_h + self.vocab_size) * std
        self.params["bo"] = np.zeros((self.n_h, 1))

        # output
        self.params["Wv"] = np.random.randn(self.vocab_size, self.n_h) * \
                            (1.0 / np.sqrt(self.vocab_size))
        self.params["bv"] = np.zeros((self.vocab_size, 1))

        # -----initialise gradients and Adam parameters-----#
        self.grads = {}
        self.adam_params = {}

        for key in self.params:
            self.grads["d" + key] = np.zeros_like(self.params[key])
            self.adam_params["m" + key] = np.zeros_like(self.params[key])
            self.adam_params["v" + key] = np.zeros_like(self.params[key])

        self.smooth_loss = -np.log(1.0 / self.vocab_size) * self.seq_len
        return

    def sigmoid(self, x):
        """
        Smoothes out values in the range of [0,1]
        """
        return 1 / (1 + np.exp(-x))

    def softmax(self, x):
        """
        Normalizes output into a probability distribution
        """
        e_x = np.exp(x - np.max(x))  # max(x) subtracted for numerical stability
        return e_x / np.sum(e_x)

    def clip_grads(self):
        """
        Limits the magnitude of gradients to avoid exploding gradients
        """
        for key in self.grads:
            np.clip(self.grads[key], -5, 5, out=self.grads[key])
        return

    def reset_grads(self):
        """
        Resets gradients to zero before each backpropagation
        """
        for key in self.grads:
            self.grads[key].fill(0)
        return

    def update_params(self, batch_num):
        """
        Updates parameters with Adam
        """
        for key in self.params:
            self.adam_params["m" + key] = self.adam_params["m" + key] * self.beta1 + \
                                          (1 - self.beta1) * self.grads["d" + key]
            self.adam_params["v" + key] = self.adam_params["v" + key] * self.beta2 + \
                                          (1 - self.beta2) * self.grads["d" + key] ** 2

            m_correlated = self.adam_params["m" + key] / (1 - self.beta1 ** batch_num)
            v_correlated = self.adam_params["v" + key] / (1 - self.beta2 ** batch_num)
            self.params[key] -= self.lr * m_correlated / (np.sqrt(v_correlated) + 1e-8)
        return

    def sample(self, h_prev, c_prev, sample_size):
        """
        Outputs a sample sequence from the model
        """
        x = np.zeros((self.vocab_size, 1))
        h = h_prev
        c = c_prev
        sample_string = ""

        for t in range(sample_size):
            y_hat, _, h, _, c, _, _, _, _ = self.forward_step(x, h, c)

            # get a random index within the probability distribution of y_hat(ravel())
            idx = np.random.choice(range(self.vocab_size), p=y_hat.ravel())
            x = np.zeros((self.vocab_size, 1))
            x[idx] = 1

            # find the char with the sampled index and concat to the output string
            char = self.idx_to_char[idx]
            sample_string += char
        return sample_string

    def forward_step(self, x, h_prev, c_prev):
        """
        Implements the forward propagation for one time step
        """
        z = np.row_stack((h_prev, x))

        f = self.sigmoid(np.dot(self.params["Wf"], z) + self.params["bf"])
        i = self.sigmoid(np.dot(self.params["Wi"], z) + self.params["bi"])
        c_bar = np.tanh(np.dot(self.params["Wc"], z) + self.params["bc"])

        c = f * c_prev + i * c_bar
        o = self.sigmoid(np.dot(self.params["Wo"], z) + self.params["bo"])
        h = o * np.tanh(c)

        v = np.dot(self.params["Wv"], h) + self.params["bv"]
        y_hat = self.softmax(v)
        return y_hat, v, h, o, c, c_bar, i, f, z

    def backward_step(self, y, y_hat, dh_next, dc_next, c_prev, z, f, i, c_bar, c, o, h):
        """
        Implements the backward propagation for one time step
        """
        dv = np.copy(y_hat)
        dv[y] -= 1  # yhat - y

        self.grads["dWv"] += np.dot(dv, h.T)
        self.grads["dbv"] += dv

        dh = np.dot(self.params["Wv"].T, dv)
        dh += dh_next

        do = dh * np.tanh(c)
        da_o = do * o * (1 - o)
        self.grads["dWo"] += np.dot(da_o, z.T)
        self.grads["dbo"] += da_o

        dc = dh * o * (1 - np.tanh(c) ** 2)
        dc += dc_next

        dc_bar = dc * i
        da_c = dc_bar * (1 - c_bar ** 2)
        self.grads["dWc"] += np.dot(da_c, z.T)
        self.grads["dbc"] += da_c

        di = dc * c_bar
        da_i = di * i * (1 - i)
        self.grads["dWi"] += np.dot(da_i, z.T)
        self.grads["dbi"] += da_i

        df = dc * c_prev
        da_f = df * f * (1 - f)
        self.grads["dWf"] += np.dot(da_f, z.T)
        self.grads["dbf"] += da_f

        dz = (np.dot(self.params["Wf"].T, da_f)
              + np.dot(self.params["Wi"].T, da_i)
              + np.dot(self.params["Wc"].T, da_c)
              + np.dot(self.params["Wo"].T, da_o))

        dh_prev = dz[:self.n_h, :]
        dc_prev = f * dc
        return dh_prev, dc_prev

    def forward_backward(self, x_batch, y_batch, h_prev, c_prev):
        """
        Implements the forward and backward propagation for one batch
        """
        x, z = {}, {}
        f, i, c_bar, c, o = {}, {}, {}, {}, {}
        y_hat, v, h = {}, {}, {}

        # Values at t= - 1
        h[-1] = h_prev
        c[-1] = c_prev

        loss = 0
        for t in range(self.seq_len):
            x[t] = np.zeros((self.vocab_size, 1))
            x[t][x_batch[t]] = 1

            y_hat[t], v[t], h[t], o[t], c[t], c_bar[t], i[t], f[t], z[t] = \
                self.forward_step(x[t], h[t - 1], c[t - 1])

            loss += -np.log(y_hat[t][y_batch[t], 0])

        self.reset_grads()

        dh_next = np.zeros_like(h[0])
        dc_next = np.zeros_like(c[0])

        for t in reversed(range(self.seq_len)):
            dh_next, dc_next = self.backward_step(y_batch[t], y_hat[t], dh_next,
                                                  dc_next, c[t - 1], z[t], f[t], i[t],
                                                  c_bar[t], c[t], o[t], h[t])
        return loss, h[self.seq_len - 1], c[self.seq_len - 1]

    def gradient_check(self, x, y, h_prev, c_prev, num_checks=10, delta=1e-6):
        """
        Checks the magnitude of gradients against expected approximate values
        """
        print("**********************************")
        print("Gradient check...\n")

        _, _, _ = self.forward_backward(x, y, h_prev, c_prev)
        grads_numerical = self.grads

        for key in self.params:
            print("---------", key, "---------")
            test = True

            dims = self.params[key].shape
            grad_numerical = 0
            grad_analytical = 0

            for _ in range(num_checks):  # sample 10 neurons

                idx = int(uniform(0, self.params[key].size))
                old_val = self.params[key].flat[idx]

                self.params[key].flat[idx] = old_val + delta
                J_plus, _, _ = self.forward_backward(x, y, h_prev, c_prev)

                self.params[key].flat[idx] = old_val - delta
                J_minus, _, _ = self.forward_backward(x, y, h_prev, c_prev)

                self.params[key].flat[idx] = old_val

                grad_numerical += (J_plus - J_minus) / (2 * delta)
                grad_analytical += grads_numerical["d" + key].flat[idx]

            grad_numerical /= num_checks
            grad_analytical /= num_checks

            rel_error = abs(grad_analytical - grad_numerical) / abs(grad_analytical + grad_numerical)

            if rel_error > 1e-2:
                if not (grad_analytical < 1e-6 and grad_numerical < 1e-6):
                    test = False
                    assert (test)

            print('Approximate: \t%e, Exact: \t%e =>  Error: \t%e' % (grad_numerical, grad_analytical, rel_error))
        print("\nTest successful!")
        print("**********************************\n")
        return

    def train(self, X, verbose=True):
        """
        Main method of the LSTM class where training takes place
        """
        J = []  # to store losses

        num_batches = len(X) // self.seq_len
        X_trimmed = X[: num_batches * self.seq_len]  # trim input to have full sequences

        for epoch in range(self.epochs):
            h_prev = np.zeros((self.n_h, 1))
            c_prev = np.zeros((self.n_h, 1))

            for j in range(0, len(X_trimmed) - self.seq_len, self.seq_len):
                # prepare batches
                x_batch = [self.char_to_idx[ch] for ch in X_trimmed[j: j + self.seq_len]]
                y_batch = [self.char_to_idx[ch] for ch in X_trimmed[j + 1: j + self.seq_len + 1]]

                loss, h_prev, c_prev = self.forward_backward(x_batch, y_batch, h_prev, c_prev)

                # smooth out loss and store in list
                self.smooth_loss = self.smooth_loss * 0.999 + loss * 0.001
                J.append(self.smooth_loss)

                # check gradients
                if epoch == 0 and j == 0:
                    self.gradient_check(x_batch, y_batch, h_prev, c_prev, num_checks=10, delta=1e-7)

                self.clip_grads()

                batch_num = epoch * self.epochs + j / self.seq_len + 1
                self.update_params(batch_num)

                # print out loss and sample string
                if verbose:
                    if j % 400000 == 0:
                        print('Epoch:', epoch, '\tBatch:', j, "-", j + self.seq_len,
                              '\tLoss:', round(self.smooth_loss, 2))
                        s = self.sample(h_prev, c_prev, sample_size=250)
                        print(s, "\n")

        return J, self.params

In [25]:
model = LSTM(char_to_idx, idx_to_char, vocab_size, epochs = 5, lr = 0.0005)

J, params = model.train(data)

**********************************
Gradient check...

--------- Wf ---------
Approximate: 	-2.632987e-04, Exact: 	-2.633167e-04 =>  Error: 	3.420726e-05
--------- bf ---------
Approximate: 	1.204189e-02, Exact: 	1.204184e-02 =>  Error: 	2.198308e-06
--------- Wi ---------
Approximate: 	1.379163e-04, Exact: 	1.379095e-04 =>  Error: 	2.493397e-05
--------- bi ---------
Approximate: 	2.886196e-03, Exact: 	2.886166e-03 =>  Error: 	5.215228e-06
--------- Wc ---------
Approximate: 	-2.066390e-02, Exact: 	-2.066391e-02 =>  Error: 	3.865803e-07
--------- bc ---------
Approximate: 	4.894596e-02, Exact: 	4.894591e-02 =>  Error: 	5.082054e-07
--------- Wo ---------
Approximate: 	8.100187e-06, Exact: 	8.138619e-06 =>  Error: 	2.366661e-03
--------- bo ---------
Approximate: 	6.089650e-03, Exact: 	6.089590e-03 =>  Error: 	4.897122e-06
--------- Wv ---------
Approximate: 	-1.860144e-03, Exact: 	-1.860077e-03 =>  Error: 	1.806624e-05
--------- bv ---------
Approximate: 	-6.578589e-01, Exact: 	-6.5785

In [26]:
a=" marblet semten...wo beging the suddent to chotting him anyshadd afren arain the turn, cleht stop finst shink tho sait grould. they said, set and harry, and they hut i stupped, now, as harry bewart, and deap see you landing facl, i swulted to "

In [27]:
len(a)

243